In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import glob
import time
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [8]:
transformer = transforms.Compose([
     transforms.Resize((150, 150)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5],
                          [0.5, 0.5, 0.5])
])

In [9]:
train_path='/content/drive/My Drive/real_and_fake_face'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)

In [10]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [11]:
print(classes)

['training_fake', 'training_real']


# **## CNN model**

In [13]:
class network(nn.Module):
    def __init__(self,num_classes=6):
        super(network,self).__init__()

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)

        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()

        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
                
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
                        
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output


In [14]:
model=network(num_classes=2).to(device)

In [15]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=10

In [17]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
print(train_count)

2041


In [18]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
    
    #Save the best model
    if train_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=train_accuracy

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 Train Loss: tensor(10.4137) Train Accuracy: 0.5149436550710436
Epoch: 1 Train Loss: tensor(1.6042) Train Accuracy: 0.6266536011758942
Epoch: 2 Train Loss: tensor(1.3142) Train Accuracy: 0.6668299853013229
Epoch: 3 Train Loss: tensor(0.9792) Train Accuracy: 0.7084762371386575
Epoch: 4 Train Loss: tensor(0.9429) Train Accuracy: 0.7403233708966193
Epoch: 5 Train Loss: tensor(0.5372) Train Accuracy: 0.8113669769720725
Epoch: 6 Train Loss: tensor(0.3111) Train Accuracy: 0.8726114649681529
Epoch: 7 Train Loss: tensor(0.2375) Train Accuracy: 0.9093581577658011
Epoch: 8 Train Loss: tensor(0.1734) Train Accuracy: 0.9377756001959824
Epoch: 9 Train Loss: tensor(0.1747) Train Accuracy: 0.9299363057324841
